In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
train = pd.read_csv('/data/spooky_author/train.csv')

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [5]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [7]:
xtrain[1]

'"No," he said, "oh, no a member of my family my niece, and a most accomplished woman."'

In [8]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [9]:
xtrain_tfv.shape

(17621, 15102)

In [10]:
print(xtrain_tfv[1])

  (0, 14862)	0.34673644833
  (0, 11446)	0.23785570257
  (0, 9156)	0.338570961322
  (0, 8899)	0.496405639021
  (0, 8268)	0.436413661856
  (0, 4746)	0.325982744163
  (0, 72)	0.406728425495


In [11]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.626 


In [16]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [17]:
xtrain_ctv.shape

(17621, 400266)

In [18]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.528 


In [28]:
cctv = CountVectorizer(analyzer='char_wb',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
cctv.fit(list(xtrain) + list(xvalid))
xtrain_cctv =  cctv.transform(xtrain) 
xvalid_cctv = cctv.transform(xvalid)

In [29]:
# Fitting a simple Logistic Regression on Counts
cclf = LogisticRegression(C=1.0)
cclf.fit(xtrain_cctv, ytrain)
predictions = cclf.predict_proba(xvalid_cctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.587 


In [24]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB(alpha=0.1)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.440 


In [25]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.485 


In [29]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [30]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.727 


In [31]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.782 


In [32]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.772 


In [33]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.768 


In [34]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.812 


In [35]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [36]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

In [37]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [38]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=120 ................
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................
[CV] lr__C=0.1, lr__penalty=l1, svd__n_components=180 ................
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=120 ................
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=120 ................
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=180 ................
[CV] lr__C=0.1, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=-0.7793279569484086, total=   7.0s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=120, score=-0.7765930373419657, total=   7.1s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=120 ................


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.6s


[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=120, score=-0.7760035233449482, total=   7.6s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=120, score=-0.7720406538319361, total=   7.8s
[CV] lr__C=1.0, lr__penalty=l1, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=180, score=-0.7440307584900074, total=   8.4s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l2, svd__n_components=180, score=-0.7431030183787967, total=   9.8s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=120 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=180, score=-0.7468123331068901, total=  10.7s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 ................
[CV]  lr__C=0.1, lr__penalty=l1, svd__n_components=180, score=-0.7471784463418926, total=  12.7s
[CV] lr__C=1.0, lr__penalty=l2, svd__n_components=180 .........

[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.2s


[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.7794151138156151, total=   6.1s
[CV] lr__C=10, lr__penalty=l1, svd__n_components=120 .................
[CV]  lr__C=1.0, lr__penalty=l1, svd__n_components=120, score=-0.7694004350727072, total=   8.2s
[CV] lr__C=10, lr__penalty=l1, svd__n_components=180 .................
[CV]  lr__C=1.0, lr__penalty=l2, svd__n_components=120, score=-0.7758637324350928, total=   5.6s
[CV] lr__C=10, lr__penalty=l1, svd__n_components=180 .................


...

[CV]  lr__C=10, lr__penalty=l2, svd__n_components=120, score=-0.7738629767284915, total=   5.0s
[CV]  lr__C=10, lr__penalty=l1, svd__n_components=180, score=-0.7369097822546963, total=   9.9s
[CV]  lr__C=10, lr__penalty=l1, svd__n_components=180, score=-0.748443602590143, total=  10.3s


[Parallel(n_jobs=-1)]: Done  21 out of  24 | elapsed:   26.4s remaining:    3.8s


[CV]  lr__C=10, lr__penalty=l2, svd__n_components=120, score=-0.7771697384864157, total=   4.8s
[CV]  lr__C=10, lr__penalty=l2, svd__n_components=180, score=-0.7446263638832216, total=   5.5s
[CV]  lr__C=10, lr__penalty=l2, svd__n_components=180, score=-0.7362018347253217, total=   5.5s


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   28.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   28.0s finished


Best score: -0.735
Best parameters set:
	lr__C: 1.0
	lr__penalty: 'l2'
	svd__n_components: 180


In [40]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.001 .................................................
[CV] nb__alpha=0.01 ..................................................
[CV] nb__alpha=0.01 ..................................................
[CV] nb__alpha=0.1 ...................................................
[CV] nb__alpha=0.1 ...................................................
[CV] ....... nb__alpha=0.001, score=-0.6204702511359115, total=   0.0s
[CV] ....... nb__alpha=0.001, score=-0.6414540936460588, total=   0.0s
[CV] ........ nb__alpha=0.01, score=-0.5107778556718855, total=   0.0s
[CV] nb__alpha=1 .....................................................
[CV] nb__alpha=1 .....................................................
[CV] nb__alpha=10 ....................................................
[CV] nb__alpha=100 ...................................................
[CV] ......... nb

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0970s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.2s finished


In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/data/glove/glove.840B.300d.txt', 'r', encoding='utf8')

for line in f:
    values = line.split()
    word = values[0]
    print(values[1:])
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))